<a href="https://colab.research.google.com/github/hydemi/curso-data-science/blob/main/ValidaRDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U openai langchain-community pypdf langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.3/567.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [2]:
from google.colab import userdata
open_ai_key = userdata.get('open_ai_key')

In [3]:
from openai import OpenAI
client = OpenAI(api_key=open_ai_key)

In [4]:
# Importing necessary modules
import os
#from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
#from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

Loading the document to the notebook

In [ ]:
# Load the file from https://www.bmj.com/content/331/7531/1498.full.pdf to this enviroment
#!wget https://www.bmj.com/content/331/7531/1498.full.pdf

--2024-07-22 03:11:17--  https://www.bmj.com/content/331/7531/1498.full.pdf
Resolving www.bmj.com (www.bmj.com)... 104.18.32.115, 172.64.155.141
Connecting to www.bmj.com (www.bmj.com)|104.18.32.115|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.bmj.com/content/bmj/331/7531/1498.full.pdf [following]
--2024-07-22 03:11:17--  https://www.bmj.com/content/bmj/331/7531/1498.full.pdf
Reusing existing connection to www.bmj.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘1498.full.pdf.1’

1498.full.pdf.1         [ <=>                ]  92.85K  --.-KB/s    in 0.01s   

2024-07-22 03:11:17 (6.56 MB/s) - ‘1498.full.pdf.1’ saved [95077]



In [ ]:
# Loading flie to PyPDFLoader
#from langchain.document_loaders import PyPDFLoader
#from langchain.chat_models import ChatOpenAI


#loader = PyPDFLoader("/content/1498.full.pdf.1")
#pages = loader.load_and_split()

# import os
# from langchain.document_loaders import PyPDFLoader

# def load_pdfs_from_directory(directory):
#     """Loads all PDF files from the specified directory into a list of LangChain Documents."""
#     all_documents = []
#     for filename in os.listdir(directory):
#         if filename.endswith(".pdf"):
#             filepath = os.path.join(directory, filename)
#             loader = PyPDFLoader(filepath)
#             pages = loader.load_and_split()
#             all_documents.extend(pages)
#     return all_documents

# # Call the function to load the documents
# directory_path = "/content/drive/MyDrive/Colab Notebooks/Integridade/"
# documents = load_pdfs_from_directory(directory_path)

In [ ]:
from langchain.prompts import PromptTemplate

text = """

"""
# Define a template for the prompt
prompt_template = """Você é um analista de mudanças, especialista em avaliar Requisições de Mudanças de uma instituição.
                  Para validar, você irá considerar o guia de instruções com o que deve ser validado em cada campo.
                  Você receberá perguntas para verificar se o preenchimento de campos específicos da Requisição de Mudança (RDM) está correto conforme o guia de instruções.
                  Você receberá a informação para validação com o campo a que se refere e o conteúdo que foi informado pelo usuário.
                  Você deverá responder o mais fielmente possível às instruções de validação.
                  Caso o conteúdo fornecido
                  Caso não informem a que campo se refere, responda "Você não informou qual campo quer que eu valide".
                  Caso não informem o conteúdo do campo, responda "Você não informou o conteúdo a ser validado".
                  Caso o campo informado não seja encontrado no Guia de instruções, responda "Não tenho conhecimento de como validar este campo"
              No texto abaixo estão as instruções do Guia de Instruções:
              ```{text}```
              """
prompt = PromptTemplate.from_template(prompt_template)

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 53.7 MB/s eta 0:00:00


In [ ]:
# Create an embedding for the documents
embeddings = OpenAIEmbeddings(openai_api_key=open_ai_key)

# Create a vector store to facilitate retrieval
vector_store = FAISS.from_documents(documents, embeddings)

# Set up the retriever from the vector store
retriever = vector_store.as_retriever()

# Initialize the LLM (you will need an API key for this)
llm = OpenAI(openai_api_key=open_ai_key, temperature=0.7)  # You can adjust the temperature as needed

# Create the QA chain using the retriever and the language model
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# Example function to ask questions
def ask_question(question):
    """Uses the QA chain to answer a given question."""
    result = qa_chain({"query": question})
    return result

# Example usage
question = "Qual é o objetivo dos documentos carregados?"
response = ask_question(question)
print(response['result'])

<ipython-input-23-abd584f61a96>:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


 O objetivo dos documentos carregados é estabelecer as regras e diretrizes da Política de Integridade e Combate à Corrupção, com o objetivo de prevenir e combater atos corruptos no âmbito do Banco.


In [ ]:
# Example usage
question = """Leia o caso e assinale a alternativa correta.

Ao selecionar fornecedores para a sua área, seu gestor pediu que você contratasse uma empresa indicada por ele. Ao consultar algumas informações sobre essa empresa, você acabou descobrindo que a proprietária é casada com o irmão do seu gestor.

Diante dessa situação, o que você faria?

a) Ficaria quieto e não falaria nada, afinal trata-se do seu gestor e ele sabe o que está fazendo.
b) Comentaria informalmente com colegas da equipe. Quem sabe a notícia se espalhe e alguém tome alguma providência.
c) Realizaria uma denúncia relatando o ocorrido para que a contratação dessa empresa seja investigada.
"""
response = ask_question(question)
print(response['result'])

 c) Realizaria uma denúncia relatando o ocorrido para que a contratação dessa empresa seja investigada.


In [ ]:
# Example usage
question = """Selecione a alternativa correta.

Com relação às denúncias de irregularidades, julgue a sentença se verdadeira ou falsa: "As denúncias, cujo denunciante poderá ser identificado ou anônimo, serão recebidas pela Ouvidoria do DF e encaminhadas para apuração pela Corregedoria, nos casos de indícios de irregularidades em processos e em caso de indícios de irregularidades envolvendo colaboradores do BRB.
Justifique a resposta
"""
response = ask_question(question)
print(response['result'])

 A sentença é parcialmente verdadeira. As denúncias serão recebidas pela Ouvidoria do Banco e encaminhadas para apuração pela Superintendência de Auditoria, em casos de indícios de irregularidades em processos e para a Corregedoria, em caso de indícios de irregularidades envolvendo colaboradores do BRB. Portanto, a sentença omitiu a informação sobre a Superintendência de Auditoria receber as denúncias.


In [ ]:
#print(response)

{'query': 'Selecione a alternativa correta.\n\nCom relação às denúncias de irregularidades, julgue a sentença se verdadeira ou falsa: "As denúncias, cujo denunciante poderá ser identificado ou anônimo, serão recebidas pela Ouvidoria do Banco e encaminhadas para apuração pela Corregedoria, nos casos de indícios de irregularidades em processos e em caso de indícios de irregularidades envolvendo colaboradores do BRB.\n', 'result': ' Verdadeira', 'source_documents': [Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Integridade/codigo_de_conduta_do_brb_06072021.pdf', 'page': 8}, page_content='#00 Pública  \nCódigo 1.007/0006  \nCódigo de Conduta do BRB – PRESI/COREG  \n \n \nVigência 06/07/2021  9/25 \nCorregedoria                                                                                                                     #00 Pública  \n3. DA DENÚNCIA  \n \n3.1. Nos termos do D ecreto n° 36.462/2015, o canal de denúncias internas e externas do BRB \né o OuvDF e po

In [ ]:
# Example usage
question = """O Código de Conduta do BRB estabelece deveres com relação ao uso, divulgação e sigilo de informações. Com relação a esses deveres, marque a opção incorreta.
a) Não disseminar qualquer tipo de informação que possa conter conteúdo calunioso ou difamatório.
b) Não comunicar qualquer informação privilegiada, que possa vir a se tornar ato ou fato relevante, ao superior imediato.
c) Não permitir o acesso de terceiros a sistemas de informações, operações e bancos de dados de responsabilidade e/ou propriedade do BRB, salvo se expressamente autorizado pelo gestor competente.
d) Fornecer informações, nos casos legais e de determinação judicial, antecedido de orientação da área jurídica do BRB.
"""
response = ask_question(question)
print(response['result'])

 b) Não comunicar qualquer informação privilegiada, que possa vir a se tornar ato ou fato relevante, ao superior imediato.


In [ ]:
# Example usage
question = """
Devido à pandemia, em um determinado dia faltaram vários colaboradores da Agência 999, restando trabalhando somente o gerente geral, o gerente de negócios e um estagiário.
Devido à necessidade de viabilizar o atendimento aos clientes, o gerente de negócios repassou seu login e senha de acesso aos sistemas para que o estagiário realizasse o cadastro dos clientes.
Com relação à situação, analise as afirmações abaixo e selecione a opção correta.

I. O gerente de negócios agiu corretamente, uma vez que o Código de Conduta diz que o colaborar deve agir, efetivamente, para o atingimento de objetivos e metas estabelecidos nos níveis operacional, tático e estratégico.
II. A conduta do gerente de negócios foi irregular, uma vez que é vedado repassar para outro logins e senhas, de caráter individual e intransferível.
III. A conduta do gerente de negócios pode ser considerada correta, desde que autorizado por seu superior hierárquico.
a) I e III são verdadeiras. x
b) Todas são verdadeiras.x
c) I e II são falsas. x
X --> d) Somente a II é verdadeira.
e) Todas são falsas.x
"""
response = ask_question(question)
print(response['result'])

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4373 tokens (4117 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
# Example usage
question = """
O Código de Conduta do BRB estabelece que o relacionamento com clientes, fornecedores, parceiros, unidades do Conglomerado BRB, órgãos/entidades públicas e demais instituições deve ser pautado pelos valores éticos e socialmente responsáveis, estabelecidos pelo BRB, evitando-se situações que possam caracterizar conflito de interesses, devendo ser observadas algumas condutas. Assim, entre as informadas abaixo, analise-as se verdadeiras ou falsas, e selecione a opção correta.


	I. Nos contatos profissionais com representantes de fornecedores, prestadores de serviços, entidades e empresas patrocinadas ou clientes, o colaborador deve estar obrigatoriamente acompanhado por um segundo colaborador, indicado pela chefia imediata e, quando membro estatutário, por outro membro estatutário ou colaborador.
	II. Em casos de interesse do Banco, o colaborador pode impor barreiras, critérios ou procedimentos que dificultem a extinção da relação contratual relativa a produtos e serviços, bem como para a transferência de relacionamento para outra instituição a pedido do cliente.
	III. É dever do colaborador do Conglomerado BRB, no que couber, observar as premissas relacionadas a conflito de interesses, previstas no normativo, no relacionamento com/entre as empresas em que o BRB participe; as áreas de risco e de negócios; na gestão de recursos próprios e na gestão de recursos de terceiros; empresas contratadas, profissionais contratados ou cedidos para o BRB.

QUAIS DOS ITENS SÃO VERDADEIROS?
"""
response = ask_question(question)
print(response['result'])

 
I. Nos contatos profissionais com representantes de fornecedores, prestadores de serviços, entidades e empresas patrocinadas ou clientes, o colaborador deve estar obrigatoriamente acompanhado por um segundo colaborador, indicado pela chefia imediata e, quando membro estatutário, por outro membro estatutário ou colaborador. 
III. É dever do colaborador do Conglomerado BRB, no que couber, observar as premissas relacionadas a conflito de interesses, previstas no normativo, no relacionamento com/entre as empresas em que o BRB participe; as áreas de risco e de negócios; na gestão de recursos próprios e na gestão de recursos de terceiros; empresas contratadas, profissionais contratados ou cedidos para o BRB.


In [ ]:
# Example usage
question = """
É vedado ao colaborador o exercício de atividade paralela, exceto de:
a) professor ou cargo ou emprego privativo de profissional da saúde com profissões regulamentadas.
b) consultor financeiro, independentemente da função ou unidade de lotação, incluindo-se nesse conceito agente de investimentos, corretor de bolsa de valores, analista de mercado, coach financeiro e demais profissionais de orientação e gestão de investimentos financeiros, exceto no interesse do Banco.
c) corretor de seguros, independentemente da função ou unidade de lotação; e corretor de imóveis, independentemente da função ou unidade de lotação.
d) sócio, empregado, consultor ou administrador de construtora/incorporadora, independentemente do cargo, carreira profissional, função ou unidade de lotação.
"""
response = ask_question(question)
print(response['result'])

 a) professor ou cargo ou emprego privativo de profissional da saúde com profissões regulamentadas.


In [ ]:
# Example usage
question = """
Nosso Código de Ética estabelece os padrões, os valores e os princípios éticos que adotamos na execução diária de nossas atividades. Todos nós, sejamos empregados, colaboradores e administradores, trabalhando no BRB ou nas empresas do Grupo, atuamos com base nos princípios e valores definidos nesse normativo, quais sejam: respeito; honestidade; compromisso; transparência; e responsabilidade, configurados nas situações abaixo descritas. De acordo com os princípios, marque a sequência correta.

a) Compromisso, responsabilidade, honestidade, respeito e transparência.
b) Responsabilidade, respeito, honestidade, compromisso e transparência.
c) Responsabilidade, respeito, transparência, compromisso e honestidade.
d) Compromisso, responsabilidade, honestidade, transparência e respeito.
"""
response = ask_question(question)
print(response['result'])
print("c) Responsabilidade, respeito, transparência, compromisso e honestidade.")

 c) Responsabilidade, respeito, transparência, compromisso e honestidade.


In [ ]:
# Example usage
question = """
Com relação ao princípio da honestidade, julgue o item a seguir: é vedada a solicitação de doações, contribuições de bens materiais ou valores a parceiros comerciais ou institucionais em nome do BRB, sob qualquer pretexto.

a) Falso
b) Verdadeiro
"""
response = ask_question(question)
print(response['result'])



b) Verdadeiro


In [ ]:
# Example usage
question = """
Sobre o valor ético da RESPONSABILIDADE, julgue o item a seguir: uma das formas de preservar os valores do BRB é a proteção contra qualquer forma de represália ou discriminação profissional a quem denunciar as violações do Código de Ética do BRB.
a) Verdadeiro
b) Falso
"""
response = ask_question(question)
print(response['result'])

 a) Verdadeiro


In [ ]:
# Example usage
question = """
Com relação ao valor ético de COMPROMISSO, julgue o item a seguir: o Banco incentiva a busca pelo equilíbrio apropriado entre trabalho, a família e a sociedade, em geral, de modo a se alcançar a felicidade, o bem-estar profissional e pessoal.
a) Verdadeiro
b) Falso
"""
response = ask_question(question)
print(response['result'])

 a) Verdadeiro


In [ ]:
# Example usage
question = """De acordo com o Código Penal Brasileiro, a corrupção pode ser Ativa ou Passiva.
Como amostra, relatamos a seguinte situação:
o condutor de veículo é parado em uma blitz, e por haver alguma irregularidade,
oferece certa quantia ao policial que o aborda, na intenção de que aquele possa
se omitir de seu exercício regular da função, e ceder-lhe vantagem indevida.
Considerando os parâmetros que definem os dois tipos de corrupção,
responda qual é o tipo de corrupção cometido por cada um."""
response = ask_question(question)
print(response['result'])

resposta = response['result']

question = f"""
Essa é a resposta de uma pergunta anterior: {resposta}.
Considerando os parâmetros que definem os dois tipos de corrupção Ativa e Passiva, avalie, marcando a opção correta:
a) A proposta do condutor do veículo é um exemplo de corrupção PASSIVA, e o policial, ao aceitar o suborno, está a praticar corrupção ATIVA.
b) A proposta do condutor do veículo é um exemplo de corrupção ATIVA, e o policial, ao aceitar o suborno, está a praticar corrupção PASSIVA.
c) Se o motorista oferece o suborno e o policial o aceita, ambos estarão a praticar corrupção ATIVA.
d) Se o motorista oferece o suborno e o policial o aceita, ambos estarão a praticar corrupção PASSIVA.
e) Nenhuma das anteriores.
"""
#response = ask_question(question)
#print(response['result'])


Nesse caso, o condutor de veículo está cometendo o crime de corrupção ativa, pois é ele quem oferece a vantagem indevida ao policial. Já o policial, caso aceite a oferta e se omita de seu dever de fiscalizar e aplicar a lei, estará cometendo o crime de corrupção passiva. 


In [ ]:
# Example usage
question = """A vedação da prática do nepotismo no âmbito do Poder Executivo Federal foi regulamentada por meio do decreto nº 7.203, de 4 de junho de 2010. No BRB, o assunto é tratado no código de conduta, o qual contempla de forma clara as definições e proibições. assim, analise as regras contidas nos textos abaixo quanto à veracidade e responsda verdadeiro ou falso:
I - É vedado nomear para o exercício de Função Gratificada, colaborador familiar da autoridade competente pela designação, ou ter colaborador familiar com ou sem função gratificada sob sua subordinação direta, inclusive na eventualidade.
"""
response = ask_question(question)
print(response['result'])

question = """A vedação da prática do nepotismo no âmbito do Poder Executivo Federal foi regulamentada por meio do decreto nº 7.203, de 4 de junho de 2010. No BRB, o assunto é tratado no código de conduta, o qual contempla de forma clara as definições e proibições. assim, analise as regras contidas nos textos abaixo quanto à veracidade e responsda verdadeiro ou falso:
II - É considerado familiar o cônjuge, o companheiro ou o parente em linha reta ou colateral, por consanguinidade ou afinidade, até o terceiro grau.
"""
response = ask_question(question)
print(response['result'])

question = """A vedação da prática do nepotismo no âmbito do Poder Executivo Federal foi regulamentada por meio do decreto nº 7.203, de 4 de junho de 2010. No BRB, o assunto é tratado no código de conduta, o qual contempla de forma clara as definições e proibições. assim, analise as regras contidas nos textos abaixo quanto à veracidade e responsda verdadeiro ou falso:
III - É vedada a nomeação de familiar para o exercício de função gratificada, mediante o ajuste de designações recíprocas, inclusive nas empresas do Conglomerado.
"""
response = ask_question(question)
print(response['result'])




 Verdadeiro
 Verdadeiro.
 Verdadeiro. O texto menciona expressamente que é vedada a nomeação de familiar para o exercício de função gratificada, mesmo mediante o ajuste de designações recíprocas, o que demonstra a preocupação em evitar qualquer tipo de favorecimento ou nepotismo nas empresas do conglomerado BRB.
